# Honorable Mentions
We insert our former experimental models that kicked off our research of the best model we could create.
More in detail, we present here:
* Our best performing attempt as a custom model
* Another attempt at transfer learning and fine tuning but exploiting VGG19 instead of Xception

Both of them share the same Data Augmentation routine that the straight forward Xception transfer model uses, so we report here only the functions building the actual model, while omitting the data pre-processing although it has been performed.

## Custom Model

In [ ]:
# Custom model made with only maxpoolings and convolutions, also add some regularization in order to achieve better performances
# There is a batch normalization after each convolution and before the relu
# There is a dropout layer after every maxpooling with 0.2

epochs = 200
input_shape = (224, 224, 3)

def build_model(input_shape):

    #input layer of the CNN
    input_layer = tfkl.Input(shape=input_shape, name='Input')


    # Convolutional part
    conv1 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        kernel_initializer='he_uniform'
    )(input_layer)

    bn1 = tfkl.BatchNormalization()(conv1)

    relu1 = tfkl.Activation('relu')(bn1)

    pool1 = tfkl.MaxPooling2D(
        pool_size=2,
        strides=3
    )(relu1)

    dp1 = tfkl.Dropout(0.2)(pool1)

    conv2 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        kernel_initializer='he_uniform'
    )(dp1)

    bn2 = tfkl.BatchNormalization()(conv2)

    relu2 = tfkl.Activation('relu')(bn2)

    conv3 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        activation='relu',
        kernel_initializer='he_uniform'
    )(relu2)

    bn3 = tfkl.BatchNormalization()(conv3)

    relu3 = tfkl.Activation('relu')(bn3)

    pool2 = tfkl.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(relu3)

    dp2 = tfkl.Dropout(0.2)(pool2)

    conv4 = tfkl.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        strides=(1, 1),
        activation='relu',
        kernel_initializer='he_uniform'
    )(dp2)

    bn4 = tfkl.BatchNormalization()(conv4)

    relu4 = tfkl.Activation('relu')(bn4)

    pool3 = tfkl.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(relu4)

    dp3 = tfkl.Dropout(0.2)(pool3)


    # Global average pooling in order to prepare data to feed the FC part
    global_average = tfkl.GlobalAveragePooling2D()(dp3)


    # FC part 
    dense1 = tfkl.Dense(units=128, activation='relu', kernel_initializer='he_uniform')(global_average)

    bn6 = tfkl.BatchNormalization()(dense1)

    dp4 = tfkl.Dropout(0.5)(bn6)
    
    rff = RandomFourierFeatures(output_dim=4096, scale=10.0, kernel_initializer="gaussian")(dp4)

    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer='he_uniform',
                              name='Output')(rff)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')

    # Return the model
    return model


model = build_model(input_shape)
model.summary()

# Create folders and callbacks
locals_callbacks = create_folders_and_callbacks(model_name='Model-v0')


# Train the model
history = model.fit(
    training_set,
    epochs=epochs,
    validation_data=validation_set,
    callbacks=locals_callbacks
).history

# Save best epoch model
model.save("/kaggle/working")

## VGG19 Transfer Learning

In [ ]:
# Transfer learning with VGG19
from tensorflow.keras.applications.vgg19 import VGG19

epochs = 150
input_shape = (224, 224, 3)

# Import VGG19 pretrained application without the FC part

base_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=8,
    classifier_activation='softmax'
)

# Freeze every layer in the net
base_model.trainable = False

def build_model(input_shape):
    # Use the VGG19 feature extraction part
    x = base_model.output

    #Build a custom FC part
    global_average = tfkl.GlobalAveragePooling2D()(x)

    dense1 = tfkl.Dense(units=512, activation='relu', kernel_initializer='he_uniform')(global_average)

    bn6 = tfkl.BatchNormalization()(dense1)

    dp4 = tfkl.Dropout(0.5)(bn6)

    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer='he_uniform',
                              name='Output')(dp4)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=base_model.input, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')

    # Return the model
    return model


model = build_model(input_shape)
model.summary()


# Create folders and callbacks and fit
locals_callbacks = create_folders_and_callbacks(model_name='VGG19_0')


# Train the model
history = model.fit(
	training_set,
	epochs=epochs,
	validation_data=validation_set,
	callbacks=locals_callbacks
).history

# Fine tuning part

# Unfreeze number of layer in the VGG19
for layer in model.layers[:3]:
    layer.trainable = False
for layer in model.layers[3:]:
    layer.trainable = True

# Recompile the model with very low learning rate
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

# Callbacks
locals_callbacks1 = create_folders_and_callbacks(model_name='VGG19_0')

# Retrain the model
history = model.fit(
	training_set,
	epochs=epochs,
	validation_data=validation_set,
	callbacks=locals_callbacks1
).history

# Save model
model.save("/kaggle/output/working/VGG19_0")